In [1]:
import Pkg;
Pkg.activate(joinpath(@__DIR__, ".."));
Pkg.instantiate();
using DelimitedFiles
using CSV
using LinearAlgebra
using ForwardDiff
using RobotDynamics
using Ipopt
using MathOptInterface
const MOI = MathOptInterface
using Random;

  Activating environment at `~/planar_quadruped_landing_main/Project.toml`


In [2]:
include("quadratic_cost.jl")
include("planar_quadruped.jl")
include("nlp.jl")
include("moi.jl")
include("costs.jl")
include("constraints.jl")
include("ref_traj.jl");

WebIO._IJuliaInit()

In [3]:
h = 1
v_init_y = sqrt(2 * 9.81 * h)

4.4294469180700204

In [4]:
# Dynamics model
model = PlanarQuadruped()
g, mb, lb, l1, l2 = model.g, model.mb, model.lb, model.l1, model.l2

# Some parameters
dt = 0.009
N = 61
times = range(0, dt * (N - 1), length=N)
k_trans = 21
n = 15
m = 5

# Initial condition. Currently, we assume the initial mode ID is 1
xinit = zeros(n)
xinit[1] = -lb / 2.5                # xb
xinit[2] = sqrt(l1^2 + l2^2) + 0.1  # yb
xinit[3] = -30 * pi / 180           # theta
xinit[6] = -lb                      # x2
xinit[7] = 0.2                      # y2

xinit[9] = -v_init_y                # yb_dot
xinit[10] = -pi/2                   # theta_dot
xinit[14] = -1.0                    # y2_dot

init_mode = 1

# Desired final state
xterm = zeros(n)
xterm[1] = -lb / 2           # xb
xterm[2] = sqrt(l1^2 + l2^2) # yb
xterm[6] = -lb;              # x2

In [5]:
# Reference Trajectory
Xref, Uref = reference_trajectory(model, N, k_trans, xterm, init_mode, dt);

In [6]:
# Objective
Q = Diagonal([1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 1.0; 0.0])
R = Diagonal([1e-3, 1e-2, 1e-3, 1e-2, 0.0])
Qf = Diagonal([10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 10.0; 0.0])

obj = map(1:N-1) do k
    LQRCost(Q, R, Xref[k], Uref[k])
end
push!(obj, LQRCost(Qf, R * 0, Xref[N], Uref[1]));

In [7]:
# Define the NLP
nlp = HybridNLP(model, obj, init_mode, k_trans, N, xinit, xterm);

In [8]:
# initialize guess
Xguess = [zeros(n) for x in Xref]
k_trans = nlp.k_trans

for k = 1:N
    if k <= k_trans
        Xguess[k] = xinit + (xterm - xinit) / (k_trans - 1) * (k - 1)
    else
        Xguess[k][1:14] = xterm[1:14]
    end
end

for k = 1:N-1
    if k < k_trans
        Xguess[k+1][end] = Xguess[k][end] + 0.001
    else
        Xguess[k+1][end] = Xguess[k][end] + 0.02
    end
end

In [10]:
Z0 = packZ(nlp, Xguess, Uref)

Z_sol, solver = solve(Z0, nlp, c_tol=1e-3, tol=1e-3)

Creating NLP Block Data...
Creating Ipopt...
Adding constraints...
starting Ipopt Solve...
This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:  1253880
Number of nonzeros in inequality constraint Jacobian.:    74115
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     1215
                     variables with only lower bounds:      120
                variables with lower and upper bounds:      121
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1032
Total number of inequality constraints...............:       61
        inequality constraints with only lower bounds:       61
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:       

([-0.2000000000000086, 0.4535533905932812, -0.5235987755982986, -1.8028182279818631e-9, -6.387775796570697e-16, -0.5000000000000108, 0.20000000000000046, -2.3088073303483943e-15, -4.429446918070019, -1.5707963267948968  …  -0.49999999999996814, -2.5668209188946017e-19, 6.3427841246588304e-15, 3.2242720501188765e-16, -3.1375156388116127e-16, 0.0, 0.0, 0.0, 0.0, 0.8447793957554794], Ipopt.Optimizer)

In [11]:
Z_sol[1:15] - xinit

15-element Vector{Float64}:
 -8.576472865229334e-15
  7.382983113757291e-15
  2.220446049250313e-16
 -1.8028182279818631e-9
 -6.387775796570697e-16
 -1.0769163338864018e-14
  4.440892098500626e-16
 -2.3088073303483943e-15
  1.7763568394002505e-15
 -2.220446049250313e-16
  0.0
  0.0
 -6.008797189593533e-16
  4.440892098500626e-16
 -1.0978238928283265e-17

In [12]:
Z_sol[end-14:end] - xterm

15-element Vector{Float64}:
 -2.3869795029440866e-14
 -8.326672684688674e-16
  3.0931899578947735e-16
  0.0
 -5.862829697035215e-19
  3.186340080674199e-14
 -2.5668209188946017e-19
  6.3427841246588304e-15
  3.2242720501188765e-16
 -3.1375156388116127e-16
  0.0
  0.0
  0.0
  0.0
  0.8447793957554794

In [13]:
@show F1y_final = Z_sol[end-18]
@show F2y_final = Z_sol[end-16]
@show F1y_final + F2y_final

F1y_final = Z_sol[end - 18] = 48.99550897182947
F2y_final = Z_sol[end - 16] = 49.10449102817053
F1y_final + F2y_final = 98.1


98.1

In [ ]:
theta = zeros(N)
for k = 1:N
    theta[k] = Z_sol[3+20*(k-1)]
end
@show theta;

In [14]:
dt = zeros(N-1)
for k = 1:N-1
    @show k
    @show dt[k] = Z_sol[20 + 20 * (k-1)]    
end

k = 1
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.01977023468436981
k = 2
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.006700201225717796
k = 3
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.002288813388934485
k = 4
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.0015211824429936362
k = 5
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.0011496006227529616
k = 6
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.0013608127154711063
k = 7
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.0012681641922323334
k = 8
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001235726466780621
k = 9
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001218664469064777
k = 10
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.0012031930840570484
k = 11
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001191658183871351
k = 12
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.00118344819160183
k = 13
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001178740148907724
k = 14
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.0011737398001027786
k = 15
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.0011741318423653957
k = 16
dt[k] = Z_sol[20 + 20 * (k - 1)] = 0.001175012362536

In [ ]:
# display(Z_sol)
writedlm("data_5.csv", Z_sol, ',')